In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional

In [2]:
data = pd.read_csv('./Data/2018-06-06-ss.cleaned.csv')

# Convert protein structures to one-hot encoded matrices
def one_hot_encode(protein):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    one_hot = np.zeros((len(protein), len(amino_acids)))
    for i, aa in enumerate(protein):
        if aa in amino_acids:
            one_hot[i, amino_acids.index(aa)] = 1
    return one_hot

sequences = data.iloc[:,2].apply(one_hot_encode)
structures = data.iloc[:,4].apply(one_hot_encode)

# print the first 10 sequences
print(sequences[:10])
print(structures[:10])

0    [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
2    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
3    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
4    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
5    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
6    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
7    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
8    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
9    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
Name: seq, dtype: object
0    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
2    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
4    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
5    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
6    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
7    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [ ]:
#Longest sequence is 5037
max_length = 5037
#Pad sequences with zeros
padded_sequences = sequences.apply(lambda x: np.pad(x, ((0, max_length - len(x)), (0, 0)), mode='constant'))

#Convert into 3D vector
X = np.stack(padded_sequences.values)

#Convert secondary structure into categorical labels
Y = pd.get_dummies(structures)